# 어텐션
## 어텐션의 구조
Decoder 개선 1

In [1]:
import numpy as np

T, H = 5, 4
hs = np.random.randn(T, H)
a = np.array([0.8, 0.1, 0.03, 0.05, 0.02])

ar = a.reshape(5,1).repeat(4, axis=1) ## repeat를 사용하지 않아도 broadcasting이 일어날 것이지만 눈에 잘 띄지 않음. 역전파도 수행해야 함. repeat에 대한 역전파는 각 미분 계수의 누적합으로 계산됨
print(f"ar: {ar.shape}")
print()

t = hs * ar

c = np.sum(t, axis=0)
print(f"c: {c.shape}")

ar: (5, 4)

c: (4,)


미니배치용 가중합

In [3]:
N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
a = np.random.randn(N, T)
ar = a.reshape(N, T, 1).repeat(H, axis=2)
ar = a.reshape(N, T, 1) # 브로드캐스트를 사용하는 경우

t = hs * ar
print(f"t: {t.shape}")

c = np.sum(t, axis=1)
print(f"c :{c.shape}")

t: (10, 5, 4)
c :(10, 4)


In [4]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None

    def forward(self, hs, a):
        N, T, H = hs.shape

        ar = a.reshape(N, T, 1).repeat(H, axis=2) ## (N, T, 1) -> (N, T, H)
        t = hs * ar ## (N, T, H)
        c = np.sum(t, axis=1) ## (N, T, H) -> (N, H)

        self.cache = (hs, ar)
        return c
    
    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape

        ## dc: (N, H) -> (N, 1, H) -> (N, T, H)
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)
        dar = dt * hs ## (N, T, H)
        dhs = dt * ar ## (N, T, H)
        da = np.sum(dar, axis=2) ## (N, T, H) -> (N, T)

        return dhs, da